In [1]:
# # # the dependencies here are a mess, I would love to have these all in the environment.yml file, but I don't know how to pass the "--no-deps" flag to pip install from the environment.yml file.
# !pip uninstall -y apache-beam
# !pip install pip install 'apache-beam[gcp]'

# !pip uninstall -y pangeo-forge-runner pangeo-forge-recipes

# # !pip install git+https://github.com/pangeo-forge/pangeo-forge-runner.git@arbitrary-injections --no-deps #TODO: this one needs to be rebased on current main
# !pip install git+https://github.com/pangeo-forge/pangeo-forge-runner.git@main --no-deps #i have deactivated the injection code for now
# !pip install git+https://github.com/jbusecke/pangeo-forge-recipes.git@dynamic_chunks_2 --no-deps
# !pip install ruamel.yaml --no-deps


In [2]:
# # test bigquery logic (WTF? Why is this working in the other notebook but not here)
# from feedstock.recipe import urls_from_bq
# urls_from_bq('CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r3i1p1f1.day.pr.gn.v20190603')

In [3]:
# global input args
repo = "https://github.com/jbusecke/cmip6-leap-feedstock.git"
ref = "beam-dataflow-testing"
# f_name = "config.json"
# test locally 
f_name = "config_local.json"

In [4]:
def shorten_id(iid):
    iid_stripped = ''.join(iid.lower().split('.')[3:])
    assert len(iid_stripped)<66
    return iid_stripped

In [5]:
# from the integration tests of pangeo-forge-runner
#https://github.com/pangeo-forge/pangeo-forge-runner/blob/main/tests/integration/test_dataflow_integration.py

import subprocess
import os
import time
import xarray as xr
import json
import random
import string

# using randomchoice due to this https://stackoverflow.com/questions/13484726/safe-enough-8-character-short-unique-random-string
alphabet = string.ascii_lowercase + string.digits

# # check written stores (should this just be
# iids = []
# with open('iids.txt') as f:
#     for line in f:
#         if not '#' in line:
#             iids.append(str(line).replace(" ", "").replace("',\n", "").replace("'","")) # FIXME: This seems ugly haha
# print(iids)

# iids = [
#     'CMIP6.CMIP.IPSL.IPSL-CM6A-LR.historical.r1i1p1f1.Omon.zmeso.gn.v20180803',
#     # 'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp585.r5i1p1f1.day.sfcWind.gn.v20210907'
# ]

# tims full request (minus the miroc ones that were not available

# ec-earth3ssp585r140i1p1f1daypslgrv2020041

iids = [
#     'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r17i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r2i3p1f1.day.pr.gn.v20201222',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r25i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r127i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r138i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r15i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r134i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r110i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r1i3p1f1.day.pr.gn.v20201222',
#  'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r4i3p1f1.day.pr.gn.v20201222',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r27i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r103i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r1i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r142i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r124i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r29i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r7i1p1f2.day.pr.gr.v20210218',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r2i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r4i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r15i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r111i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r126i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r136i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r20i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r122i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r131i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r7i1p1f2.day.sfcWind.gr.v20210218',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r24i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r1i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r133i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r13i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r141i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r19i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r30i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r18i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r123i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r8i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r121i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r23i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r146i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r16i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r2i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r149i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r23i1p1f1.day.psl.gr.v20200407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r145i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r29i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.MOHC.HadGEM3-GC31-MM.ssp585.r2i1p1f3.day.psl.gn.v20200515',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r12i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r23i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r7i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r7i1p1f2.day.psl.gr.v20210218',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r14i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r132i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r9i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r118i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r21i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r13i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r115i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r26i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r129i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r6i1p1f2.day.sfcWind.gr.v20210517',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r12i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r113i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r106i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r15i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r12i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r114i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r6i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r25i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r128i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r12i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r17i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r147i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r17i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r27i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r4i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r21i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r101i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r130i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r2i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r150i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r19i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r17i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r108i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r144i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r21i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r25i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r10i1p1f2.day.psl.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r120i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r19i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r5i3p1f1.day.pr.gn.v20201222',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r5i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r125i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r18i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r29i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r28i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r10i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r1i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r112i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r8i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r30i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r30i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r10i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r143i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r139i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r10i1p1f2.day.sfcWind.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r8i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r137i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r19i1p1f2.day.sfcWind.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r23i1p1f2.day.pr.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r140i1p1f1.day.psl.gr.v20200412', # fails with depenency issues? (only intermittent)
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r148i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r27i1p1f2.day.psl.gr.v20201015',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r119i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r105i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r8i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r116i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r11i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r15i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r20i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r22i1p1f2.day.pr.gr.v20210407',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r117i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r107i1p1f1.day.psl.gr.v20200412',
#  'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r3i3p1f1.day.pr.gn.v20201222',
#  'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp245.r16i1p1f1.day.pr.gn.v20200623',
#  'CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r4i1p1f1.day.pr.gn.v20190603',
#  'CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r5i1p1f1.day.pr.gn.v20190603',
#  'CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r2i1p1f1.day.pr.gn.v20190603',
 'CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r3i1p1f1.day.pr.gn.v20190603',
       ]


# iids = [
#     # 'CMIP6.ScenarioMIP.MOHC.HadGEM3-GC31-MM.ssp585.r2i1p1f3.day.psl.gn.v20200515',
#     # 'CMIP6.ScenarioMIP.MOHC.HadGEM3-GC31-MM.ssp585.r3i1p1f3.day.psl.gn.v20200507',
#     # 'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r140i1p1f1.day.psl.gr.v20200412', # fails with depenency issues?
#     # 'CMIP6.CMIP.IPSL.IPSL-CM5A2-INCA.historical.r1i1p1f1.Omon.zmeso.gn.v20200729',
# ]

# ## Testing 'throwing mem at the problem'
# high_mem_iids = [
#     'CMIP6.CMIP.NCC.NorESM2-LM.historical.r1i1p1f1.Omon.umo.gn.v20190815',
#     'CMIP6.CMIP.IPSL.IPSL-CM6A-LR.historical.r1i1p1f1.Omon.zmeso.gn.v20180803',
# ]
# iids = high_mem_iids
# # I know these failed with high mem use :https://console.cloud.google.com/dataflow/jobs/us-central1/2023-03-29_16_00_29-11923246449479218691;graphView=0?project=leap-pangeo&authuser=2&pageState=(%22dfTime%22:(%22l%22:%22dfJobMaxTime%22))
# # and https://console.cloud.google.com/dataflow/jobs/us-central1/2023-03-29_16_29_49-6440994980175925686;graphView=0?project=leap-pangeo&authuser=2&pageState=(%22dfTime%22:(%22l%22:%22dfJobMaxTime%22))
# # lets see if these beast machines can solve this issue?
# # maybe Ryans PR will alleviate this? It seems to do!

my_env = os.environ.copy()
my_env["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/juliusbusecke/Downloads/keys/leap-pangeo-dataflow_service_account.json"

def submit_job(repo, ref, f_name, iid, jobname, machine_type):
    inject_dict = {
        "get_iid": {"iid":iid}, 
        "get_jobname":{"jobname": jobname}
    }

    cmd = [
        "pangeo-forge-runner",
        "bake",
        "--repo",
        repo,
        "--ref",
        ref,
        "--json",
        "-f",
        f_name,
        "--Bake.inject",
        json.dumps(inject_dict),
        "--Bake.job_name",
        jobname,
        "--DataflowBakery.machine_type", # i hope this is just ignored in the local case?
        machine_type,
    ]
    print("\nCurrent environment:")
    

    print("\nSubmitting job...")
    print(f"{' '.join(cmd)}")
    submit_proc = subprocess.run(cmd, capture_output=True, env=my_env)
    return submit_proc

def print_stdout(submit_proc):
    for line in submit_proc.stdout.decode().splitlines():
        json_line = json.loads(line)
        for k in json_line.keys():
            for subline in json_line[k].splitlines():
                print(subline)

def print_stderr(submit_proc):
    for line in submit_proc.stderr.decode().splitlines():
        print(line)



for iid in iids:
    print(f"""Submitting ({iid})""")
    unique_iid = iid+'.'+''.join(random.choices(alphabet, k=8))
    jobname=shorten_id(unique_iid)
    # machine_type = "e2-highmem-16" # Ill have to experiment with that
    # machine_type = "e2-highmem-8" # seems i had some other errors here...
    # As far as I can see in the recent 'job metrics', we never go above 1 core. So we only need the beast machines for the larger models
    # machine_type = "e2-standard-2"
    machine_type = "n2-highmem-2"
    # if iid in high_mem_iids:
    #     machine_type = "n2-highmem-16"
    # there are definitely some jobs failing with high mem useage. TODO: I need a way to override this for some of them...
    
    try:
        # check environment

        submit_proc = submit_job(repo, ref, f_name, iid, jobname, machine_type)
        assert submit_proc.returncode == 0
        lastline = json.loads(submit_proc.stdout.decode().splitlines()[-1])
        assert lastline["status"] == "submitted"
        job_id = lastline["job_id"]
        job_name = lastline["job_name"]
        
        print(f"Job submitted with {job_id = }")
        with open('submitted_iids.txt', 'a') as f:
            f.write(iid+'\n')
    except Exception as err:
        print(f'Submission FAILED with {err}')
    print('STDOUT')
    print_stdout(submit_proc)
    print('STDERR')
    print_stderr(submit_proc)

Submitting (CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r3i1p1f1.day.pr.gn.v20190603)

Current environment:

Submitting job...
pangeo-forge-runner bake --repo https://github.com/jbusecke/cmip6-leap-feedstock.git --ref beam-dataflow-testing --json -f config_local.json --Bake.inject {"get_iid": {"iid": "CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r3i1p1f1.day.pr.gn.v20190603"}, "get_jobname": {"jobname": "mri-esm2-0historicalr3i1p1f1dayprgnv20190603fojesrli"}} --Bake.job_name mri-esm2-0historicalr3i1p1f1dayprgnv20190603fojesrli --DataflowBakery.machine_type n2-highmem-2


KeyboardInterrupt: 

In [ ]:
len(iids)

129

In [ ]:
# for line in submit_proc.stderr.decode().splitlines():
#     # print(json.loads(line)['message'])
#     print(line)

In [ ]:
# err

In [6]:
import xarray as xr

In [7]:
xr.open_dataset('/Users/juliusbusecke/Code/CMIP6-LEAP-feedstock/local_storage/target/CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r3i1p1f1.day.pr.gn.v20190603.zarr', engine='zarr')

FileNotFoundError: No such file or directory: '/Users/juliusbusecke/Code/CMIP6-LEAP-feedstock/local_storage/target/CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r3i1p1f1.day.pr.gn.v20190603.zarr'

In [10]:
str = 'Error during running: Traceback (most recent call last):\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1423, in process\n    return self.do_fn_invoker.invoke_process(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 839, in invoke_process\n    self._invoke_process_per_window(\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 985, in _invoke_process_per_window\n    self.process_method(*args_for_process, **kwargs_for_process),\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/transforms/core.py\", line 1961, in <lambda>\n    wrapper = lambda x, *args, **kwargs: [fn(x, *args, **kwargs)]\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_recipes/transforms.py\", line 71, in wrapper\n    result = func(item, **kwargs)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_recipes/openers.py\", line 32, in open_url\n    cache.cache_file(url, secrets, **kw)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_recipes/storage.py\", line 188, in cache_file\n    _copy_btw_filesystems(input_opener, target_opener)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_recipes/storage.py\", line 38, in _copy_btw_filesystems\n    data = source.read(BLOCK_SIZE)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/implementations/http.py\", line 600, in read\n    return super().read(length)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/spec.py\", line 1790, in read\n    out = self.cache._fetch(self.loc, self.loc + length)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/caching.py\", line 379, in _fetch\n    self.cache = self.fetcher(start, bend)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/asyn.py\", line 121, in wrapper\n    return sync(self.loop, func, *args, **kwargs)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/asyn.py\", line 106, in sync\n    raise return_result\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/asyn.py\", line 61, in _runner\n    result[0] = await coro\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/implementations/http.py\", line 669, in async_fetch_range\n    out = await r.read()\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/aiohttp/client_reqrep.py\", line 1037, in read\n    self._body = await self.content.read()\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/aiohttp/streams.py\", line 375, in read\n    block = await self.readany()\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/aiohttp/streams.py\", line 397, in readany\n    await self._wait(\"readany\")\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/aiohttp/streams.py\", line 304, in _wait\n    await waiter\naiohttp.client_exceptions.ClientPayloadError: Response payload is not completed\n\nDuring handling of the above exception, another exception occurred:\n\nTraceback (most recent call last):\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/sdk_worker.py\", line 297, in _execute\n    response = task()\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/sdk_worker.py\", line 372, in <lambda>\n    lambda: self.create_worker().do_instruction(request), request)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/sdk_worker.py\", line 625, in do_instruction\n    return getattr(self, request_type)(\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/sdk_worker.py\", line 663, in process_bundle\n    bundle_processor.process_bundle(instruction_id))\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/bundle_processor.py\", line 1040, in process_bundle\n    input_op_by_transform_id[element.transform_id].process_encoded(\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/bundle_processor.py\", line 232, in process_encoded\n    self.output(decoded_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/operations.py\", line 570, in output\n    _cast_to_receiver(self.receivers[output_index]).receive(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/operations.py\", line 264, in receive\n    self.consumer.process(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/operations.py\", line 952, in process\n    delayed_applications = self.dofn_runner.process(o)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1425, in process\n    self._reraise_augmented(exn, windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1513, in _reraise_augmented\n    raise exn\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1423, in process\n    return self.do_fn_invoker.invoke_process(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 625, in invoke_process\n    self.output_handler.handle_process_outputs(\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1607, in handle_process_outputs\n    self._write_value_to_tag(tag, windowed_value, watermark_estimator)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1720, in _write_value_to_tag\n    self.main_receivers.receive(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/operations.py\", line 264, in receive\n    self.consumer.process(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/operations.py\", line 952, in process\n    delayed_applications = self.dofn_runner.process(o)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1425, in process\n    self._reraise_augmented(exn, windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1513, in _reraise_augmented\n    raise exn\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1423, in process\n    return self.do_fn_invoker.invoke_process(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 625, in invoke_process\n    self.output_handler.handle_process_outputs(\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1607, in handle_process_outputs\n    self._write_value_to_tag(tag, windowed_value, watermark_estimator)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1720, in _write_value_to_tag\n    self.main_receivers.receive(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/operations.py\", line 264, in receive\n    self.consumer.process(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/operations.py\", line 952, in process\n    delayed_applications = self.dofn_runner.process(o)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1425, in process\n    self._reraise_augmented(exn, windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1513, in _reraise_augmented\n    raise exn\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1423, in process\n    return self.do_fn_invoker.invoke_process(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 625, in invoke_process\n    self.output_handler.handle_process_outputs(\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1607, in handle_process_outputs\n    self._write_value_to_tag(tag, windowed_value, watermark_estimator)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1720, in _write_value_to_tag\n    self.main_receivers.receive(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/operations.py\", line 264, in receive\n    self.consumer.process(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/operations.py\", line 952, in process\n    delayed_applications = self.dofn_runner.process(o)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1425, in process\n    self._reraise_augmented(exn, windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1533, in _reraise_augmented\n    raise new_exn\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1423, in process\n    return self.do_fn_invoker.invoke_process(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 839, in invoke_process\n    self._invoke_process_per_window(\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 985, in _invoke_process_per_window\n    self.process_method(*args_for_process, **kwargs_for_process),\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/transforms/core.py\", line 1961, in <lambda>\n    wrapper = lambda x, *args, **kwargs: [fn(x, *args, **kwargs)]\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_recipes/transforms.py\", line 71, in wrapper\n    result = func(item, **kwargs)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_recipes/openers.py\", line 32, in open_url\n    cache.cache_file(url, secrets, **kw)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_recipes/storage.py\", line 188, in cache_file\n    _copy_btw_filesystems(input_opener, target_opener)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_recipes/storage.py\", line 38, in _copy_btw_filesystems\n    data = source.read(BLOCK_SIZE)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/implementations/http.py\", line 600, in read\n    return super().read(length)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/spec.py\", line 1790, in read\n    out = self.cache._fetch(self.loc, self.loc + length)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/caching.py\", line 379, in _fetch\n    self.cache = self.fetcher(start, bend)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/asyn.py\", line 121, in wrapper\n    return sync(self.loop, func, *args, **kwargs)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/asyn.py\", line 106, in sync\n    raise return_result\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/asyn.py\", line 61, in _runner\n    result[0] = await coro\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/implementations/http.py\", line 669, in async_fetch_range\n    out = await r.read()\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/aiohttp/client_reqrep.py\", line 1037, in read\n    self._body = await self.content.read()\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/aiohttp/streams.py\", line 375, in read\n    block = await self.readany()\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/aiohttp/streams.py\", line 397, in readany\n    await self._wait(\"readany\")\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/aiohttp/streams.py\", line 304, in _wait\n    await waiter\naiohttp.client_exceptions.ClientPayloadError: Response payload is not completed [while running 'Create|OpenURLWithFSSpec|OpenWithXarray|KeepOnlyVariableId|StoreToZarr/OpenURLWithFSSpec/Open with fsspec']\n", "exc_info": "Traceback (most recent call last):\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/bin/pangeo-forge-runner\", line 8, in <module>\n    sys.exit(main())\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_runner/cli.py\", line 28, in main\n    app.start()\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_runner/cli.py\", line 23, in start\n    super().start()\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/traitlets/config/application.py\", line 473, in start\n    return self.subapp.start()\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_runner/commands/bake.py\", line 263, in start\n    pipeline.run()\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/pipeline.py\", line 577, in run\n    return self.runner.run_pipeline(self, self._options)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/direct/direct_runner.py\", line 129, in run_pipeline\n    return runner.run_pipeline(pipeline, options)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/portability/fn_api_runner/fn_runner.py\", line 202, in run_pipeline\n    self._latest_run_result = self.run_via_runner_api(\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/portability/fn_api_runner/fn_runner.py\", line 224, in run_via_runner_api\n    return self.run_stages(stage_context, stages)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/portability/fn_api_runner/fn_runner.py\", line 455, in run_stages\n    bundle_results = self._execute_bundle(\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/portability/fn_api_runner/fn_runner.py\", line 783, in _execute_bundle\n    self._run_bundle(\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/portability/fn_api_runner/fn_runner.py\", line 1012, in _run_bundle\n    result, splits = bundle_manager.process_bundle(\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/portability/fn_api_runner/fn_runner.py\", line 1387, in process_bundle\n    raise RuntimeError(result.error)\nRuntimeError: Traceback (most recent call last):\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1423, in process\n    return self.do_fn_invoker.invoke_process(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 839, in invoke_process\n    self._invoke_process_per_window(\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 985, in _invoke_process_per_window\n    self.process_method(*args_for_process, **kwargs_for_process),\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/transforms/core.py\", line 1961, in <lambda>\n    wrapper = lambda x, *args, **kwargs: [fn(x, *args, **kwargs)]\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_recipes/transforms.py\", line 71, in wrapper\n    result = func(item, **kwargs)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_recipes/openers.py\", line 32, in open_url\n    cache.cache_file(url, secrets, **kw)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_recipes/storage.py\", line 188, in cache_file\n    _copy_btw_filesystems(input_opener, target_opener)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_recipes/storage.py\", line 38, in _copy_btw_filesystems\n    data = source.read(BLOCK_SIZE)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/implementations/http.py\", line 600, in read\n    return super().read(length)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/spec.py\", line 1790, in read\n    out = self.cache._fetch(self.loc, self.loc + length)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/caching.py\", line 379, in _fetch\n    self.cache = self.fetcher(start, bend)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/asyn.py\", line 121, in wrapper\n    return sync(self.loop, func, *args, **kwargs)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/asyn.py\", line 106, in sync\n    raise return_result\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/asyn.py\", line 61, in _runner\n    result[0] = await coro\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/implementations/http.py\", line 669, in async_fetch_range\n    out = await r.read()\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/aiohttp/client_reqrep.py\", line 1037, in read\n    self._body = await self.content.read()\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/aiohttp/streams.py\", line 375, in read\n    block = await self.readany()\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/aiohttp/streams.py\", line 397, in readany\n    await self._wait(\"readany\")\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/aiohttp/streams.py\", line 304, in _wait\n    await waiter\naiohttp.client_exceptions.ClientPayloadError: Response payload is not completed\n\nDuring handling of the above exception, another exception occurred:\n\nTraceback (most recent call last):\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/sdk_worker.py\", line 297, in _execute\n    response = task()\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/sdk_worker.py\", line 372, in <lambda>\n    lambda: self.create_worker().do_instruction(request), request)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/sdk_worker.py\", line 625, in do_instruction\n    return getattr(self, request_type)(\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/sdk_worker.py\", line 663, in process_bundle\n    bundle_processor.process_bundle(instruction_id))\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/bundle_processor.py\", line 1040, in process_bundle\n    input_op_by_transform_id[element.transform_id].process_encoded(\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/bundle_processor.py\", line 232, in process_encoded\n    self.output(decoded_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/operations.py\", line 570, in output\n    _cast_to_receiver(self.receivers[output_index]).receive(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/operations.py\", line 264, in receive\n    self.consumer.process(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/operations.py\", line 952, in process\n    delayed_applications = self.dofn_runner.process(o)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1425, in process\n    self._reraise_augmented(exn, windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1513, in _reraise_augmented\n    raise exn\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1423, in process\n    return self.do_fn_invoker.invoke_process(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 625, in invoke_process\n    self.output_handler.handle_process_outputs(\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1607, in handle_process_outputs\n    self._write_value_to_tag(tag, windowed_value, watermark_estimator)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1720, in _write_value_to_tag\n    self.main_receivers.receive(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/operations.py\", line 264, in receive\n    self.consumer.process(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/operations.py\", line 952, in process\n    delayed_applications = self.dofn_runner.process(o)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1425, in process\n    self._reraise_augmented(exn, windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1513, in _reraise_augmented\n    raise exn\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1423, in process\n    return self.do_fn_invoker.invoke_process(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 625, in invoke_process\n    self.output_handler.handle_process_outputs(\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1607, in handle_process_outputs\n    self._write_value_to_tag(tag, windowed_value, watermark_estimator)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1720, in _write_value_to_tag\n    self.main_receivers.receive(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/operations.py\", line 264, in receive\n    self.consumer.process(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/operations.py\", line 952, in process\n    delayed_applications = self.dofn_runner.process(o)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1425, in process\n    self._reraise_augmented(exn, windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1513, in _reraise_augmented\n    raise exn\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1423, in process\n    return self.do_fn_invoker.invoke_process(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 625, in invoke_process\n    self.output_handler.handle_process_outputs(\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1607, in handle_process_outputs\n    self._write_value_to_tag(tag, windowed_value, watermark_estimator)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1720, in _write_value_to_tag\n    self.main_receivers.receive(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/operations.py\", line 264, in receive\n    self.consumer.process(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/worker/operations.py\", line 952, in process\n    delayed_applications = self.dofn_runner.process(o)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1425, in process\n    self._reraise_augmented(exn, windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1533, in _reraise_augmented\n    raise new_exn\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 1423, in process\n    return self.do_fn_invoker.invoke_process(windowed_value)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 839, in invoke_process\n    self._invoke_process_per_window(\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/runners/common.py\", line 985, in _invoke_process_per_window\n    self.process_method(*args_for_process, **kwargs_for_process),\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/apache_beam/transforms/core.py\", line 1961, in <lambda>\n    wrapper = lambda x, *args, **kwargs: [fn(x, *args, **kwargs)]\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_recipes/transforms.py\", line 71, in wrapper\n    result = func(item, **kwargs)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_recipes/openers.py\", line 32, in open_url\n    cache.cache_file(url, secrets, **kw)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_recipes/storage.py\", line 188, in cache_file\n    _copy_btw_filesystems(input_opener, target_opener)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/pangeo_forge_recipes/storage.py\", line 38, in _copy_btw_filesystems\n    data = source.read(BLOCK_SIZE)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/implementations/http.py\", line 600, in read\n    return super().read(length)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/spec.py\", line 1790, in read\n    out = self.cache._fetch(self.loc, self.loc + length)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/caching.py\", line 379, in _fetch\n    self.cache = self.fetcher(start, bend)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/asyn.py\", line 121, in wrapper\n    return sync(self.loop, func, *args, **kwargs)\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/asyn.py\", line 106, in sync\n    raise return_result\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/asyn.py\", line 61, in _runner\n    result[0] = await coro\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/fsspec/implementations/http.py\", line 669, in async_fetch_range\n    out = await r.read()\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/aiohttp/client_reqrep.py\", line 1037, in read\n    self._body = await self.content.read()\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/aiohttp/streams.py\", line 375, in read\n    block = await self.readany()\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/aiohttp/streams.py\", line 397, in readany\n    await self._wait(\"readany\")\n  File \"/Users/juliusbusecke/miniconda/envs/cmip6-leap-feedstock/lib/python3.9/site-packages/aiohttp/streams.py\", line 304, in _wait\n    await waiter\naiohttp.client_exceptions.ClientPayloadError: Response payload is not completed [while running 'Create|OpenURLWithFSSpec|OpenWithXarray|KeepOnlyVariableId|StoreToZarr/OpenURLWithFSSpec/Open with fsspec']\n'

SyntaxError: invalid syntax (2036255239.py, line 1)